<a href="https://colab.research.google.com/github/kacperkurowski/CE888/blob/master/lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lab 5

##Loading Files

In [51]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv", header = None)
print(df)

arr = df.values
print(arr.shape)

rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]
(24983, 101)
1835357 (18

Replacing 10% Values with 99

In [52]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])


new_arr, idx = replace(arr, 0.1)

print(arr[idx[0][0], idx[1][0]])
print(new_arr[idx[0][0], idx[1][0]])



-4.66
99.0


LFM of the gidden ratings

In [53]:
n_latent_factors = 2

user_ratings = arr #change to new_arr to check other validation
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

#print(latent_user_preferences)
#print(latent_item_features)

def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=40):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not rating == 99.0:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()
        print(iteration) 
        if (iteration % 10) == 0:
            print(mse)

sgd()  # Note how the MSE decreases with the number of iterations

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

0
58.120942222352575
1
2
3
4
5
6
7
8
9
10
24.6100045162196
11
12
13
14
15
16
17
18
19
20
21.71689766524792
21
22
23
24
25
26
27
28
29
30
18.140832582207743
31
32
33
34
35
36
37
38
39


array([[ 7.79604196e+01, -2.64377016e+00, -3.76672958e+00, ...,
        -3.43308944e+00, -4.26191647e+00, -3.33709237e+00],
       [ 9.45134363e+01,  2.78689195e+00,  2.32500782e+00, ...,
         2.71134234e+00,  1.80905351e+00,  3.28979487e+00],
       [ 4.92483415e+01,  6.12061136e+00,  6.58075390e+00, ...,
         6.76792433e+00,  6.37764080e+00,  7.42934623e+00],
       ...,
       [ 3.90509166e+01, -6.47707967e-01, -1.10863995e+00, ...,
        -9.43566210e-01, -1.34715564e+00, -8.43306594e-01],
       [ 3.78505341e+01,  1.33488727e-01, -1.98991704e-01, ...,
        -4.12976161e-02, -4.19455973e-01,  1.14586698e-01],
       [ 7.28876522e+01,  3.11230850e+00,  2.90068623e+00, ...,
         3.19570627e+00,  2.51634963e+00,  3.71607068e+00]])

Comparing the data

In [54]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

comparison_data
# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(74.0, 77.96041962670158)","(-7.82, -2.643770164371384)","(8.79, -3.766729580830225)","(-9.66, -3.8003037328233478)","(-8.16, -5.948665207243876)","(-7.52, -2.6666302346930495)","(-8.5, -3.16032442335072)","(-9.85, -3.259855556998855)","(4.17, -2.302487396499922)","(-8.98, -5.065851531260313)","(-4.76, -2.4791372189949956)","(-8.5, -2.647182556672288)","(-6.75, -3.46544187807869)","(-7.18, -3.906951051114492)","(8.45, -3.5529797459427273)","(-7.18, -3.3376516227886697)","(-7.52, -4.870765300629657)","(-7.43, -3.37686142090031)","(-9.81, -3.136873429032917)","(-9.85, -3.1332279381323573)","(-9.85, -4.541128655750464)","(-9.37, -2.350064346312339)","(1.5, -3.8391066084293253)","(-4.37, -4.242875343977204)","(-9.81, -5.823471149282174)","(-8.5, -4.293425075932429)","(1.12, -3.5728341210378507)","(7.82, -0.41488510907425)","(2.86, -2.9934161892147815)","(9.13, -0.40778507064771236)","(-7.43, -5.469356412632932)","(2.14, -2.014474152595618)","(-4.08, -0.4449990887063276)","(-9.08, -5.873400632310561)","(7.82, -3.680974444801598)","(5.05, -0.8131769045179476)","(4.95, 0.006918548799996805)","(-9.17, -5.1445030048095814)","(-8.4, -3.0787895568801273)","(-8.4, -3.7432760061163477)",...,"(8.59, -1.9948256353018907)","(3.59, -1.0446838036725596)","(-6.84, -4.417315396558244)","(-9.03, -4.769371247853698)","(2.82, -1.9227242630547237)","(-1.36, -1.5781095321734535)","(-9.08, -5.157739402314948)","(8.3, -1.3121662629380608)","(5.68, -1.5893699427477892)","(-4.81, -4.117053295893911)","(99.0, -4.501414937398737)","(99.0, -1.591991063728221)","(99.0, -3.7323780299250937)","(99.0, -5.4982642690241645)","(99.0, -4.2025778347733995)","(99.0, -1.5901866203773432)","(99.0, -3.704250522659685)","(-9.42, -2.9537821860813325)","(99.0, -4.532712453806523)","(99.0, -3.831829623503727)","(99.0, -2.501860848272623)","(-7.72, -3.4374090880754915)","(99.0, -2.2965139488751327)","(99.0, -3.7564073484500557)","(99.0, -3.87276345409118)","(99.0, -3.309419387828688)","(99.0, -2.2036329690353553)","(99.0, -1.9211233753561683)","(99.0, -0.28271746510638945)","(99.0, -3.802066673384805)","(2.82, -1.9015607921447546)","(99.0, -3.179336125392752)","(99.0, -1.6347013761897362)","(99.0, -2.880829745109254)","(99.0, -2.8313466627941617)","(99.0, -2.6726998815135343)","(-5.63, -2.246549917369287)","(99.0, -3.4330894380327837)","(99.0, -4.261916472422724)","(99.0, -3.3370923686127627)"
1,"(100.0, 94.51343631092706)","(4.08, 2.7868919537584977)","(-0.29, 2.325007822126585)","(6.36, 2.3682404101310808)","(4.37, 0.6820453523886967)","(-2.38, 1.8980733328260229)","(-9.66, 3.8647654381884173)","(-0.73, 0.8436349549416624)","(-5.34, 0.12936564484587176)","(8.88, 1.4694078981312773)","(9.22, 3.1485719488094612)","(6.75, 4.041056418372125)","(8.64, 3.9168726333155064)","(4.42, -1.1762283151012651)","(7.43, 3.85569204864296)","(4.56, -1.351487833015076)","(-0.97, -2.8844551450050773)","(4.66, -0.2670290389565672)","(-0.68, 0.32852947176562375)","(3.3, 1.662002163705646)","(-1.21, 0.3955623045078733)","(0.87, 4.617523416603256)","(8.64, 3.179875898346708)","(8.35, 2.23569781879153)","(9.17, 0.2386162550709749)","(0.05, 2.651085791007811)","(7.57, 3.683654124467811)","(4.71, 5.433602222894196)","(0.87, 3.750026247367521)","(-0.39, 5.141663560118008)","(6.99, 1.7790649971867363)","(6.5, 4.570641800798674)","(-0.92, 5.397202402765564)","(7.14, 0.6772355104112177)","(9.03, 3.204609235669254)","(-1.8, 5.33561456727181)","(0.73, 5.44206249402666)","(7.09, 0.31864817299490195)","(3.4, 3.394820482061696)","(-0.87, 3.471958623847228)",...,"(-6.7, 4.888808804394127)","(-3.35, 5.320524985149215)","(-9.03, 2.270830810099964)","(4.47, 1.1957634078242558)","(4.08, 4.658976472450177)","(-3.83, 4.823308724396447)","(8.74, 1.124456125032805)","(1.12, 4.8566416

In [0]:
n_latent_factors = 2

user_ratings = new_arr #change to new_arr to check other validation
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

sgd()  # Note how the MSE decreases with the number of iterations

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

now we can compare the 2 data sets

last step is to run the data on the test data set
Previously the arr was a better at predicting the data.